In [5]:
import binascii
import numpy as np

In [1]:
PATH_TO_VSI = "./sample_vsi/OS-2/_OS-2_/stack1/frame_t.ets"

In [3]:
# Constants
CHAR = 1
UCHAR = 2
SHORT = 3
USHORT = 4
INT = 5
UINT = 6
LONG = 7
ULONG = 8
FLOAT = 9
DOUBLE = 10

RAW = 0
JPEG = 2
JPEG_2000 = 3
JPEG_LOSSLESS = 5
PNG = 8
BMP = 9

PIXEL_BYTE_WIDTHS = {CHAR: 1, UCHAR: 1, SHORT: 2, USHORT: 2, INT: 4, UINT: 4, LONG: 8, ULONG: 8, FLOAT: 4, DOUBLE: 8}
COMPRESSION_TYPES = {RAW: "RAW", JPEG: "JPEG", JPEG_2000: "JPEG_2000", JPEG_LOSSLESS: "JPEG_LOSSLESS", PNG: "PNG", BMP: "BMP"}

In [6]:
sis_format = None
with open(PATH_TO_VSI, 'rb') as f:
    sis_format = f.read(4).decode('utf-8').rstrip('\x00')
    header_size = int.from_bytes(f.read(4), byteorder='little')
    version = int.from_bytes(f.read(4), byteorder='little')
    dimensions = int.from_bytes(f.read(4), byteorder='little')
    additional_header_offset = int.from_bytes(f.read(8), byteorder='little')
    additional_header_size = int.from_bytes(f.read(4), byteorder='little')
    f.seek(f.tell() + 4) # reserved
    chunk_offset = int.from_bytes(f.read(8), byteorder='little')
    number_chunks = int.from_bytes(f.read(4), byteorder='little')
    f.seek(f.tell() + 4) # reserved

    f.seek(additional_header_offset)
    ets_format = f.read(4).decode('utf-8').rstrip('\x00')
    f.seek(f.tell() + 4) # extra version number

    #print(int.from_bytes(f.read(4), byteorder='little'))
    
    pixel_type = int.from_bytes(f.read(4), byteorder='little')
    ms_size_c = int.from_bytes(f.read(4), byteorder='little')
    colorspace = int.from_bytes(f.read(4), byteorder='little')
    compression_type = int.from_bytes(f.read(4), byteorder='little')
    compression_quality = int.from_bytes(f.read(4), byteorder='little')
    tile_X = int.from_bytes(f.read(4), byteorder='little')
    tile_Y = int.from_bytes(f.read(4), byteorder='little')
    tile_Z = int.from_bytes(f.read(4), byteorder='little')

    f.seek(f.tell() + 4 * 17) # pixel info hints

    color_size = ms_size_c * PIXEL_BYTE_WIDTHS[pixel_type]
    background_color = f.read(color_size).hex()

    f.seek(f.tell() + 4*10 - color_size) # background color

    component_order = int.from_bytes(f.read(4), byteorder='little')
    bgr_s = (component_order == 1 and COMPRESSION_TYPES[compression_type] == COMPRESSION_TYPES[RAW])
    use_pyramid = int.from_bytes(f.read(4), byteorder='little') != 0

    ms_rgb = ms_size_c > 1

    f.seek(chunk_offset)

    # Tile offsets is a long array of length number_chunks
    tile_offsets = np.empty(shape=(number_chunks), dtype=int)
    temp_tiles = []

    for i in range(number_chunks):
        f.seek(f.tell() + 4)
        tile_coordinate = np.empty(shape=(dimensions), dtype=int)
        for j in range(dimensions):
            tile_coordinate[j] = int.from_bytes(f.read(4), byteorder='little')
        tile_offsets[i] = int.from_bytes(f.read(8), byteorder='little')
        number_bytes = int.from_bytes(f.read(4), byteorder='little')
        f.seek(f.tell() + 4)
        temp_tiles.append(tile_coordinate)
    
    max_resolution = 0

    if use_pyramid:
        for i in temp_tiles:
            if i[len(i)-1] > max_resolution:
                max_resolution = i[len(i)-1]

    max_resolution += 1

    max_X = np.zeros(shape=(max_resolution), dtype=int)
    max_Y = np.zeros(shape=(max_resolution), dtype=int)
    max_Z = np.zeros(shape=(max_resolution), dtype=int)
    max_T = np.zeros(shape=(max_resolution), dtype=int)
    max_C = np.zeros(shape=(max_resolution), dtype=int)
    dim_order = {"T": None, "C": None, "Z": None, "X": None, "Y": None}
    rows = 0
    cols = 0
    for t in temp_tiles:
        if use_pyramid:
            resolution = t[len(t)-1]
        else:
            resolution = 0
        
        tv = dim_order["T"]
        zv = dim_order["Z"] 
        cv = dim_order["C"]

        if tv is None:
            t_index = -1
        else:
            t_index = tv + 2
        if zv is None:
            z_index = -1
        else:
            z_index = zv + 2
        if cv is None:
            c_index = -1
        else:
            c_index = cv + 2
        
        if (use_pyramid and t_index == len(t) - 1):
            tv = None
            t_index = -1
        if (use_pyramid and z_index == len(t) - 1):
            zv = None
            z_index = -1
        
        if use_pyramid:
            upper_limit = len(t) - 1
        else:
            upper_limit = len(t)

        if ((t_index < 0 or t_index >= upper_limit) 
            and (z_index < 0 or z_index >= upper_limit) 
            and (c_index < 0 or c_index >= upper_limit)):
            t_index-=1
            z_index-=1
            c_index-=1
            if "T" in dim_order:
                dim_order["T"] = t_index - 2
            if "Z" in dim_order:
                dim_order["Z"] = z_index - 2
            if "C" in dim_order:
                dim_order["C"] = c_index - 2
        
        if (tv is None and zv is None):
            if (len(t) > 4 and cv == None):
                c_index = 2
                dim_order["C"] = c_index - 2
            
            if len(t) > 4:
                if cv == None:
                    t_index = 3
                else:
                    t_index = 2 + c_index
                if t_index < len(t):
                    dim_order["T"] = t_index - 2
                else:
                    t_index = -1
            
            if (len(t) > 5):
                if (cv is None):
                    z_index = 4
                else:
                    z_index = c_index + 1
                if (z_index < len(t)):
                    dim_order["Z"] = z_index - 2
                else:
                    z_index = -1
        
        if (t[0] > max_X[resolution]):
            max_X[resolution] = t[0]
        if (t[1] > max_Y[resolution]):
            max_Y[resolution] = t[1]
        
        if (t_index >= 0 and t[t_index] > max_T[resolution]):
            max_T[resolution] = t[t_index]
        
        if (z_index >= 0 and t[z_index] > max_Z[resolution]):
            max_Z[resolution] = t[z_index]
        
        if (c_index >= 0 and t[c_index] > max_C[resolution]):
            max_C[resolution] = t[c_index]
        
    size_Z = max_Z[0] + 1

    # if (max_C[0] > 0):
    #     size_C *= max_C[0]
    
    size_T = max_T[0] + 1

    if (size_Z == 0):
        size_Z = 1
    
    image_count = size_Z * size_T

    if (max_C[0] > 0):
        image_count *= (max_C[0] + 1)
    
    if (max_Y[0] >=1):
        rows = rows + max_Y[0] + 1
    else:
        rows = rows + 1
    
    if (max_X[0] >= 1):
        cols = cols + max_X[0] + 1
    
    else:
        cols = cols + 1
    



            
    

    

    for t in temp_tiles:
        if use_pyramid:
            resolution = t[len(t)-1]
        if t[0] > max_X[resolution]:
            max_X[resolution] = t[0]
        if t[1] > max_Y[resolution]:    
            max_Y[resolution] = t[1]
        

    

np.int64(14540)

In [26]:
len(tile_offsets)

494

In [7]:
temp_tiles

[array([0, 0, 0, 0]),
 array([1, 0, 0, 0]),
 array([2, 0, 0, 0]),
 array([3, 0, 0, 0]),
 array([4, 0, 0, 0]),
 array([5, 0, 0, 0]),
 array([6, 0, 0, 0]),
 array([7, 0, 0, 0]),
 array([8, 0, 0, 0]),
 array([9, 0, 0, 0]),
 array([10,  0,  0,  0]),
 array([11,  0,  0,  0]),
 array([12,  0,  0,  0]),
 array([13,  0,  0,  0]),
 array([0, 1, 0, 0]),
 array([1, 1, 0, 0]),
 array([2, 1, 0, 0]),
 array([3, 1, 0, 0]),
 array([4, 1, 0, 0]),
 array([5, 1, 0, 0]),
 array([6, 1, 0, 0]),
 array([7, 1, 0, 0]),
 array([8, 1, 0, 0]),
 array([9, 1, 0, 0]),
 array([10,  1,  0,  0]),
 array([11,  1,  0,  0]),
 array([12,  1,  0,  0]),
 array([13,  1,  0,  0]),
 array([0, 2, 0, 0]),
 array([1, 2, 0, 0]),
 array([2, 2, 0, 0]),
 array([3, 2, 0, 0]),
 array([4, 2, 0, 0]),
 array([5, 2, 0, 0]),
 array([6, 2, 0, 0]),
 array([7, 2, 0, 0]),
 array([8, 2, 0, 0]),
 array([9, 2, 0, 0]),
 array([10,  2,  0,  0]),
 array([11,  2,  0,  0]),
 array([12,  2,  0,  0]),
 array([13,  2,  0,  0]),
 array([0, 3, 0, 0]),
 array

In [ ]:
temp_tiles

In [12]:
def intersection(region1, region2):
    x1, y1, w1, h1 = region1
    x2, y2, w2, h2 = region2

    if x1 + w1 < x2 or x2 + w2 < x1 or y1 + h1 < y2 or y2 + h2 < y1:
        return None
    else:
        x = max(x1, x2)
        y = max(y1, y2)
        w = min(x1 + w1, x2 + w2) - x
        h = min(y1 + h1, y2 + h2) - y
        return (x, y, w, h)




def decode_tile(row, col):
    













def open_bytes(x, y, w, h):
    with open(PATH_TO_VSI, 'rb') as f:
        bytes_per_pixel = PIXEL_BYTE_WIDTHS[pixel_type]
        pixel = ms_size_c * bytes_per_pixel
        output_row_len = w * pixel
        
        tile_rows = rows
        tile_cols = cols

        output_row = 0
        output_col = 0

        intersection = None

        for row in tile_rows:
            for col in tile_cols:
                width = tile_X
                height = tile_Y
                tile_region = (col * width, row*height, width, height)
                intersection = intersection((x, y, w, h), tile_region)
                if intersection is None:
                    continue
                
                intersection_X = 0

                if (tile_region[0] < x):
                    intersection_X = x - tile_region[0]

                # Code to decode tile goes here
                
                row_len = pixel * min(intersection[2], w)

                output_offset = output_row * output_row_len + output_col

                for trow in range(intersection[3]):
                    real_row = trow + intersection[1] - tile_region[1]
                    input_offset = pixel * (real_row * width + intersection_X)

                    #Code to copy data

                    output_offset += output_row_len
                
                output_col += row_len

            if (intersection is not None):
                output_row += intersection[3]
                output_col = 0
        
        # Code to convert bgr to rgb

        # Code to return bytes image

In [10]:
tile_X, tile_Y, tile_Z, max_resolution, compression_type, compression_quality, pixel_type, ms_size_c, colorspace, component_order, bgr_s, use_pyramid, ms_rgb, background_color, tile_offsets, max_resolution

(512,
 512,
 1,
 np.int64(6),
 3,
 90,
 2,
 3,
 4,
 1,
 False,
 True,
 True,
 'ffffff',
 array([    292,   14935,   30863,   42565,   56350,   70890,   82623,
          95918,  109282,  121242,  133891,  147138,  159678,  171185,
         178709,  190718,  200248,  209195,  220193,  230822,  239706,
         249588,  259441,  268612,  278093,  288020,  297750,  307196,
         313735,  325481,  334382,  343429,  352948,  362006,  369881,
         379974,  388498,  396974,  406142,  414637,  423149,  448165,
         455155,  467036,  478183,  488104,  499822,  509951,  539076,
         551289,  561405,  571584,  582091,  592774,  602893,  612820,
         620237,  632479,  641159,  650131,  659979,  669032,  678177,
         688531,  697576,  706040,  723384,  715039,  732408,  753339,
         741651,  759997,  768224,  776462,  785184,  793720,  802034,
         811597,  819978,  827604,  836019,  844414,  852495,  861884,
         868142,  897062,  906480,  916323,  926799,  937189

In [11]:
bytes_per_pixel

4

In [79]:
int.from_bytes(pixel_type, byteorder="little")

2